## MNIST (Self)

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfdt  
#tensorflow's data provider for MNIST

In [3]:
mnist_dataset, mnist_info = tfdt.load(name='MNIST',with_info=True,as_supervised = True)

# once we have loaded the dataset, we can easily extract the training and testing dataset with the built references
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']


num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

# .num_examples -> It contains the number of images in the test data set.
# mnist_info.splits['test'].num_examples -> returns 10,000, which is the number of samples (images) in the test split of the mnist data set.

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)
#

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)


BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


In [4]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                            tf.keras.layers.Dense(output_size, activation='softmax')   
                            ])

In [5]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [9]:
NUM_EPOCHS = 5  # do it 6 and see different

model.fit(train_data, epochs = NUM_EPOCHS, validation_data=(validation_inputs, validation_targets), verbose=2)

Epoch 1/5
540/540 - 2s - loss: 0.0236 - accuracy: 0.9930 - val_loss: 0.0310 - val_accuracy: 0.9917 - 2s/epoch - 4ms/step
Epoch 2/5
540/540 - 2s - loss: 0.0214 - accuracy: 0.9935 - val_loss: 0.0325 - val_accuracy: 0.9897 - 2s/epoch - 3ms/step
Epoch 3/5
540/540 - 2s - loss: 0.0202 - accuracy: 0.9938 - val_loss: 0.0292 - val_accuracy: 0.9908 - 2s/epoch - 3ms/step
Epoch 4/5
540/540 - 2s - loss: 0.0192 - accuracy: 0.9943 - val_loss: 0.0284 - val_accuracy: 0.9912 - 2s/epoch - 3ms/step
Epoch 5/5
540/540 - 2s - loss: 0.0164 - accuracy: 0.9953 - val_loss: 0.0258 - val_accuracy: 0.9913 - 2s/epoch - 3ms/step


In [10]:
test_loss, test_accuracy = model.evaluate(test_data)


1/1 [==============================] - 1s 687ms/step - loss: 0.1221 - accuracy: 0.9717


In [11]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.12. Test accuracy: 97.17%
